In [ ]:
"""
Created on Tue Mar 19 15:37 2019

Figure showing thickness evolution of snow and ice for paper

@author: m300411
"""

In [ ]:
import numpy as np
import clalib.memls_functions as memls
import clalib.simplification_functions as sf
import clalib.satsim_analysis_functions as ana
import xarray as xr
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import clalib.satsim_analysis_functions as ana
import seaborn as sns

In [ ]:
%matplotlib qt5

In [ ]:
sns.set_context('paper')

READ IN THE DATA

In [ ]:
inputpath = '/work/mh0033/m300411/SatSim/MEMLS_exp/Paper_Data/'
inputpath_samsim = '/work/mh0033/m300411/SatSim/SAMSIM/'
outputpath_fig = '/work/mh0033/m300411/SatSim/MEMLS_exp/Paper_Data/Figures/'

In [ ]:
memls_input_FYI = xr.open_dataset(inputpath+'inputMEMLS_depthidx_75N00W_exp037.nc')#.resample(time='1D').mean()
timelength_FYI = memls_input_FYI['time']
ts_month_FYI = timelength_FYI['time.month']

thick_sn_file_FYI = np.loadtxt(inputpath_samsim+'75N00W-p4/dat_snow.dat') #thick_snow,T_snow,psi_l_snow,psi_s_snow
thick_sn_FYI = thick_sn_file_FYI[:,0]
thick_sn_FYI[thick_sn_FYI==0] = np.nan

memls_input_FYI['thick_snow'] = xr.DataArray(thick_sn_FYI, coords=[original_time], dims=['time'])#.resample(time='1D').mean()

memls_input_MYI = xr.open_dataset(inputpath+'inputMEMLS_depthidx_NorthPole_exp038.nc')#.resample(time='1D').mean()
timelength_MYI = memls_input_MYI['time']
ts_month_MYI = timelength_MYI['time.month']
                              
thick_sn_file_MYI = np.loadtxt(inputpath_samsim+'NorthPole-p4/dat_snow.dat') #thick_snow,T_snow,psi_l_snow,psi_s_snow
thick_sn_MYI = thick_sn_file_MYI[:,0]
thick_sn_MYI[thick_sn_MYI==0] = np.nan

memls_input_MYI['thick_snow'] = xr.DataArray(thick_sn_MYI, coords=[original_time], dims=['time'])#.resample(time='1D').mean()

COMPUTE THE TOTAL THICKNESS FROM THE LAYERS

In [ ]:
tot_thick0 = np.zeros(len(timelength_FYI))
for tt,timet in enumerate(timelength_FYI):
    print(tt)
    a = memls_input_FYI['thickness'].sel(sens_exp='complex',time=timet).dropna(dim='depth',how='any')
    if a.size:
        tot_thick0[tt] = a.depth.max()  
    else:
        tot_thick0[tt] = np.nan  
tot_thick_FYI = xr.DataArray(tot_thick0,dims=memls_input_FYI['thickness'].sel(sens_exp='complex',depth=0.0).dims)      



In [ ]:
tot_thick1 = np.zeros(len(timelength_MYI))
for tt,timet in enumerate(timelength_MYI):
    print(tt)
    a = memls_input_MYI['thickness'].sel(sens_exp='complex',time=timet).dropna(dim='depth',how='any')
    if a.size:
        tot_thick1[tt] = a.depth.max()  
    else:
        tot_thick1[tt] = np.nan  
tot_thick_MYI = xr.DataArray(tot_thick1,dims=memls_input_MYI['thickness'].sel(sens_exp='complex',depth=0.0).dims)      
          

PLOT THE TIME EVOLUTION OF THE ICE AND SNOW HEIGHT

In [ ]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True,figsize=(8,8/2))
ax[0].plot(timelength_FYI,tot_thick_FYI*-1,'k-')
ax[0].plot(timelength_FYI,memls_input_FYI['thick_snow'],c='grey')
ax[0].axhline(y=0,c='k',linestyle=':')
fig.autofmt_xdate()

ax[1].plot(timelength_MYI,tot_thick_MYI*-1,'k-')
ax[1].plot(timelength_MYI,memls_input_MYI['thick_snow'],c='grey')
ax[1].axhline(y=0,c='k',linestyle=':')
fig.autofmt_xdate()

sns.despine()
plt.tight_layout()
fig.savefig(outputpath_fig+'snowiceevol_new.pdf',bbox_inches='tight')